# Tutorial: Combinators (Stateful `evaluate(env)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/combinator.html

This is the stateful variant: definitions are kept in a shared environment across cells.


In [1]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import EvaluationEnvironment, evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")

env = EvaluationEnvironment()
stateful = evaluate(env)


Using bqn_jax from: /home/johtok/repos/jax/xlapl/bqn-jax/src/bqn_jax/__init__.py


## Before You Start

1. Run the setup cell first to create `env` and `stateful`.
2. Run cells in order from top to bottom, because later cells may depend on earlier definitions.
3. Each `stateful("...")` call updates and reuses the same environment.
4. If results look odd after reruns, restart the kernel and run all cells again.
5. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## What's a combinator?

Combinators build new functions from existing ones without introducing explicit variables.


In [2]:
stateful('|∘- 6')


Array(6., dtype=float32, weak_type=True)

In [3]:
stateful('7 |∘- 9')


Array(2., dtype=float32, weak_type=True)

In [4]:
stateful('14‿8 |∘- 19‿6')
stateful('14‿8 +´∘|∘- 19‿6')


Array(7., dtype=float32)

## Comparisons

Use these examples to see scalar and elementwise boolean comparisons.


In [5]:
stateful('3 < 4')
stateful('4 > ∞')
stateful('∞ < @')


Array(0, dtype=int32)

In [6]:
stateful('\'e\' = "George Boole"')
stateful('+´ \'e\' = "George Boole"')
stateful('\'e\' +´∘= "George Boole"  # With a combinator')


Array(3, dtype=int32)

In [7]:
stateful('"abcd" ×´∘= "abdd"')


Array(0, dtype=int32)

In [8]:
stateful('"abcd" ≡ "abdd"')
stateful('"abc"‿"de" ≡ "abc"‿"de"')


Array(1, dtype=int32)

In [9]:
stateful('2‿3‿4‿2 ≠ 3‿3‿2‿2')
stateful('2‿3‿4‿2 ≢ 3‿3‿2‿2')


Array([4], dtype=int32)

## Length, rank, and depth

These primitives inspect structure: item count, dimensionality, and nesting depth.


In [10]:
stateful('≠ "testing"')
stateful('≠ ⟨⟩')
stateful('≠ ⟨ π, ∘, "element" ⋄ ⟨\'l\',1,5,\'t\'⟩ ⟩')
stateful('≠ 4')


Array(1, dtype=int32)

In [11]:
stateful('= 0.5')
stateful('= ↕3')
stateful("= 'a'")
stateful('= "a"')


Array(1, dtype=int32)

In [12]:
stateful('≡ "dream"                  # An ordinary dream')
stateful('≡ "d"‿"r"‿"e"‿"a"‿"m"      # What if the letters were strings?')
stateful('≡ ⟨ "d"‿"r"‿"e"‿"a"‿"m" ⟩  # We have to go deeper')


Array(3, dtype=int32)

## Composition

Composition combines smaller transforms into larger reusable pipelines.


In [13]:
stateful('(≠"string") = ≠"sting"')


Array(0, dtype=int32)

In [14]:
stateful('=´≠¨ ⟨"string","sting"⟩')


Array(0, dtype=int32)

In [15]:
stateful('"string" =○≠ "sting"')


Array(0, dtype=int32)

In [16]:
stateful('"string" ⋈○≠ "sting"')
stateful('⋈○≠ "sting"')


Array([5], dtype=int32)

In [17]:
stateful('2 ⋆⟜- 3')
stateful('2 ⋆⊸- 3')


Array(4.389056, dtype=float32, weak_type=True)

In [18]:
stateful('4 -⊸⌽ " before"  # Rotate to the right by four')
stateful('4 ⌽⁼  " before"  # Okay this time Undo is better')


Array([102, 111, 114, 101,  32,  98, 101], dtype=int32)

In [19]:
stateful('¬⊸× 0.5')


Array(0.25, dtype=float32, weak_type=True)

In [20]:
stateful('↕⊸÷ 8')
stateful('¬⊸× ↕⊸÷ 8')


Array([0.      , 0.109375, 0.1875  , 0.234375, 0.25    , 0.234375,
       0.1875  , 0.109375], dtype=float32, weak_type=True)

In [21]:
stateful('1⊸+ 5')
stateful('+⟜1 5')


Array(6., dtype=float32, weak_type=True)

In [22]:
stateful('"const"˜ 5')
stateful('@ "const"˜ 6')


Array([ 99, 111, 110, 115, 116], dtype=int32)

In [23]:
stateful('+⊸1 5')


Array(1., dtype=float32, weak_type=True)

In [24]:
stateful('↕⊸÷ 8')
stateful('(↕8) ÷ 7')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

In [25]:
stateful('-⟜1 8')


Array(7., dtype=float32, weak_type=True)

In [26]:
stateful('↕⊸÷⟜(-⟜1) 8')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

In [27]:
stateful('(↕÷-⟜1) 8')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

## Base decoding continued

This revisits decoding and progressively compresses it into tacit combinator form.


In [28]:
stateful('@ + +´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [29]:
stateful('@⊸+ +´¨ (⌽2⋆↕8)⊸×¨ -⟜\'0\' "01000010"‿"01010001"‿"01001110"')
stateful('(@⊸+)∘(+´¨)∘((⌽2⋆↕8)⊸×¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [30]:
stateful('@⊸+∘(+´∘((⌽2⋆↕8)⊸×)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [31]:
stateful('@⊸+∘(+´∘×⟜(⌽2⋆↕8)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [32]:
stateful('(((((1×10)+2)×10)+3)×10)+4')
stateful('((1×⟜10⊸+2)×⟜10⊸+3)×⟜10⊸+4   # Make the combining step a function')
stateful('4+⟜(10⊸×)3+⟜(10⊸×)2+⟜(10⊸×)1 # Flip the combining function around')
stateful("+⟜(10⊸×)´ 4‿3‿2‿1            # Now it's a BQN fold")
stateful('+⟜(10⊸×)´ ⌽ 1‿2‿3‿4          # To fold in reverse, reverse then fold')


Array(1234., dtype=float32, weak_type=True)

In [33]:
stateful('+´∘×⟜(⌽2⋆↕8) "01010001"-\'0\'')
stateful('+⟜(2⊸×)´∘⌽ "01010001"-\'0\'')


Array(81., dtype=float32)

In [34]:
stateful('@⊸+∘(+⟜(2⊸×)´∘⌽¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [35]:
stateful('(@+ ·+⟜(2⊸×)´∘⌽¨ -⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

## Summary

If any result is unclear, rerun the section above step by step before moving on.
